In [7]:
GCS_INPUT_PATH = "gs://hackathon_pjs/population"


In [11]:
SCRIPT_NAME = "upload_population_data"
OUTPUT_VERSION = 1

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)


In [12]:
!rm -r {ec2_input_path}
!rm -r {ec2_output_path}


In [13]:
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

In [14]:
!gsutil cp -r {GCS_INPUT_PATH}/*.zip {ec2_input_path}/.

Copying gs://hackathon_pjs/population/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.zip...
/ [1 files][  1.0 GiB/  1.0 GiB]                                                
Operation completed over 1 objects/1.0 GiB.                                      


In [20]:
!unzip {ec2_input_path}/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.zip -d {ec2_input_path}

Archive:  /volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.zip
   creating: /volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0/
 extracting: /volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.tif.aux.xml  
 extracting: /volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.tif  
 extracting: /volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.tfw  
 extracting: /volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0/GHSL_data_access_v1.3.pdf  
 extracting: /volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.tif.xml  
 extracting: /volum

In [21]:
import os

In [23]:
files = os.listdir(os.path.join(ec2_input_path,"GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0"))

In [35]:
source = "{}/{}/{}".format(ec2_input_path,"GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0","GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.tif")

In [26]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import MultiPolygon, Polygon

In [27]:
minx = -63
maxx = -62
miny = 18
maxy = 18.5

In [41]:
geom = Polygon([(minx,miny),
               (maxx,miny),
               (maxx,maxy),
               (minx,maxy)])

In [29]:
geoms = MultiPolygon([geom])

In [30]:
output_path = "{}/population.tif".format(ec2_output_path)

In [36]:
source

'/volumes/data/upload_population_data/input_V01/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.tif'

In [44]:
with rasterio.open(source) as src:
    meta = src.meta
    out_image, out_transform = mask(src, geoms,  crop=True) 
    #out_meta = src.meta.copy()
    #out_meta.update({"driver": "GTiff",
    #                 "height": out_image.shape[1],
    #                 "width": out_image.shape[2],
    #                 "transform": out_transform})

    """
    with rasterio.open(output_path, "w",**out_meta) as dest:
            dest.write(out_image)
    """

{'count': 1,
 'crs': CRS({'lon_0': 0, 'ellps': 'WGS84', 'no_defs': True, 'units': 'm', 'x_0': 0, 'proj': 'moll', 'y_0': 0}),
 'driver': 'GTiff',
 'dtype': 'float32',
 'height': 60829,
 'nodata': -3.4028234663852886e+38,
 'transform': Affine(250.0, 0.0, -17619594.54744353,
        0.0, -250.0, 8722279.46186849),
 'width': 141969}

In [ ]:
!gsutil cp {output_path} {GCS_INPUT_PATH}